In [4]:
import pandas as pd
import numpy as np

data_path = 'cover_data.csv'
data = pd.read_csv(data_path)

print(data.describe())
print("")
print(list(data.columns.values))

           Elevation         Aspect          Slope  \
count  581012.000000  581012.000000  581012.000000   
mean     2959.365301     155.656807      14.103704   
std       279.984734     111.913721       7.488242   
min      1859.000000       0.000000       0.000000   
25%      2809.000000      58.000000       9.000000   
50%      2996.000000     127.000000      13.000000   
75%      3163.000000     260.000000      18.000000   
max      3858.000000     360.000000      66.000000   

       Horizontal_Distance_To_Hydrology  Vertical_Distance_To_Hydrology  \
count                     581012.000000                   581012.000000   
mean                         269.428217                       46.418855   
std                          212.549356                       58.295232   
min                            0.000000                     -173.000000   
25%                          108.000000                        7.000000   
50%                          218.000000                       3

4 Possible Wilderness Areas, 40 Soil Types. 

Pre-one-hot-encoded. Can carry on as is.

## Split Data/Scale

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Feature/Label Split
features = data.iloc[:, 1:-1]
labels = data.iloc[:, -1]

# Train/Test Split & Scale
x_train, x_test, y_train, y_test = train_test_split(features, labels, random_state = 222, test_size = 0.2)

scaler = StandardScaler()
x_train_scale = scaler.fit_transform(x_train)
x_test_scale = scaler.transform(x_test)

x_train_scale = pd.DataFrame(x_train_scale, columns = x_train.columns)
x_test_scale = pd.DataFrame(x_test_scale, columns = x_test.columns)

print(x_train_scale.head())

     Aspect     Slope  Horizontal_Distance_To_Hydrology  \
0 -0.407971 -0.281543                         -1.268687   
1  1.700678  0.385828                         -1.127598   
2  0.887597 -0.681965                         -0.671412   
3 -0.944068 -0.148069                          0.447892   
4  0.744638  0.385828                         -0.868936   

   Vertical_Distance_To_Hydrology  Horizontal_Distance_To_Roadways  \
0                       -0.797237                        -0.449812   
1                       -0.694508                        -1.452329   
2                       -0.780115                         1.375629   
3                        0.743693                         0.327572   
4                       -0.523294                        -1.152151   

   Hillshade_9am  Hillshade_Noon  Hillshade_3pm  \
0       1.039892        0.186766      -0.718431   
1      -1.012311       -0.622319       0.534339   
2      -0.489932        1.147554       1.134625   
3       0.480200    

## Model Archirecture
Now scaling is complete, free to carry on with model. 

As data is tabular, with prediction categorical, a number of feed-forward Dense layers is going to be our choice. Sparse cross categorical entropy is our loss function, as labels are integers.

In [11]:
import tensorflow as tf

from sklearn.metrics import r2_score

# Create Neural Network Model
def design_model(features):
  model = tf.keras.models.Sequential(name = "my_first_model")
  #without hard-coding
  input = tf.keras.layers.InputLayer(input_shape=(features.shape[1],)) 
  #add the input layer
  model.add(input) 
  #add a hidden layer with 128 neurons
  model.add(tf.keras.layers.Dense(128, activation='relu')) 
  #add an output layer to our model
  model.add(tf.keras.layers.Dense(1)) 
  opt = tf.keras.optimizers.Adam(learning_rate=0.1)
  model.compile(loss='sparse_categorical_crossentropy',  metrics=['accuracy'], optimizer=opt)
  return model

In [12]:
# Fit and Predict
model = design_model(x_train_scale)
model.fit(x_train_scale, y_train, batch_size = 10, epochs = 40, validation_split = 0.3)
  

# Make predictions and evaluate
y_pred = model.predict(x_test_scale)

Epoch 1/40


2025-10-02 21:41:41.438579: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2025-10-02 21:41:41.466439: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] model_pruner failed: INVALID_ARGUMENT: Graph does not contain terminal node AssignAddVariableOp_2.


32537/32537 [==============================] - ETA: 0s - loss: 0.0000e+00 - accuracy: 0.0724

2025-10-02 21:44:02.259634: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


32537/32537 [==============================] - 173s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0724 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 2/40
32537/32537 [==============================] - 171s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0652 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 3/40
32537/32537 [==============================] - 173s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0652 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 4/40
32537/32537 [==============================] - 173s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0652 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 5/40
32537/32537 [==============================] - 172s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0652 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 6/40
32537/32537 [==============================] - 174s 5ms/step - loss: 0.0000e+00 - accuracy: 0.0652 - val_loss: 0.0000e+00 - val_accuracy: 0.0648
Epoch 7/40
32537/32537 [==============================] - 172s 5ms/step - l

2025-10-03 04:03:00.317018: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


3632/3632 [==============================] - 4s 1ms/step
